In [ ]:
# Import modules
import numpy as np

# Import PySwarms
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

### Variables globales del problema

In [ ]:
# Para VERTEX COVER

import networkx as nx
#G = nx.read_edgelist(r"..\vertex_cover_benchmarks\ENZYMES_g102.edges", nodetype=int)
#G = nx.read_edgelist(r"..\vertex_cover_benchmarks\aves-sparrowlyon-flock-season2-unw.edges", nodetype=int)
#G = nx.read_edgelist("brock200_1_complement.edges", nodetype=int)
#G = nx.read_edgelist(r"..\vertex_cover_benchmarks\sparrow_unweighted.edges", nodetype=int)
G = nx.read_edgelist(r"..\vertex_cover_benchmarks\cats.edges", nodetype=int)
n = G.order()

In [ ]:
# Para KNAPSACK

values = []
weights = []

with open(r"..\knapsack_benchmarks\s003.kp") as f:
    n, W = [int(x) for x in next(f).split()] # read first line
    for line in f: # read rest of lines
        nums = line.split()
        values.append(int(nums[0]))
        weights.append(int(nums[1]))


print(values)
print(weights)

### Parámetros

In [ ]:
DIMENSION = 30
SAMPLE_SIZE = 100
MAX_ITER = 5000

### Funciones de fitness

Se han transformado ligeramente porque pyswarms utiliza problemas de minimización

In [ ]:
import random

def is_cover(sol, G):
    for u, v in G.edges():
        if sol[u - 1] == 0 and sol[v - 1] == 0:
            return False
    return True

def naive_cover(sol, G):
    for u, v in G.edges():
        if sol[u - 1] == 0 and sol[v - 1] == 0:
            node = random.choice([u, v])
            sol[node - 1] = 1
    return

@ps.cost
def fitness_vertex_cover(sol):
    n = G.order()
    if is_cover(sol, G):
        return sum(sol)
    else:
        naive_cover(sol, G)
        return sum(sol)

@ps.cost
def fitness_knapsack(sol):
    n = len(sol)
    fit = 0
    wei = 0
    for i in range(n):
        fit += values[i] * sol[i]
        wei += weights[i] * sol[i]
    while wei > W:
        sacar = random.randrange(0, n)
        wei -= weights[sacar] * sol[sacar]
        fit -= values[sacar] * sol[sacar]
        sol[sacar] = 0
    return -fit

### Hiperparámetros de PSO

In [ ]:
# Se suelen elegir valores bajos para la inercia (w) y altos
# para las componentes memorísticas y sociales (c1, c2).
# k es el número de vecinos considerados y p representa la métrica Lp elegida
# (la distancia de Hamming en vectores binarios es equivalente a Manhattan o L1).
options = {'c1': 0.7, 'c2': 0.7, 'w':0.2, 'k':SAMPLE_SIZE, 'p':1}

### Resultados de N ejecuciones de VERTEX COVER

In [ ]:
# Elegir la ruta donde guardar los resultados:
output = open(r"..\results\cats\PSO.csv", "a")
# Elegir el número de ejecuciones:
num_ejec = 20
for _ in range(num_ejec):
    optimizer = ps.discrete.binary.BinaryPSO(n_particles=SAMPLE_SIZE, dimensions=n, options=options)
    cost, solution = optimizer.optimize(fitness_vertex_cover, iters=MAX_ITER)
    fit = n - int(cost)
    output.write(f'{solution},')
    output.write(f'{fit}\n')
output.close()

### Resultados de N ejecuciones de KNAPSACK

In [ ]:
# Elegir la ruta donde guardar los resultados:
output = open(r"..\results\kn3\PSO.csv", "a")
# Elegir el número de ejecuciones:
num_ejec = 20
for _ in range(num_ejec):
    optimizer = ps.discrete.binary.BinaryPSO(n_particles=SAMPLE_SIZE, dimensions=n, options=options)
    cost, solution = optimizer.optimize(fitness_knapsack, iters=MAX_ITER)
    fit = - int(cost)
    output.write(f'{solution},')
    output.write(f'{fit}\n')
output.close()